# Deep ESNs for Text Classification

This notebook provides a use case example of the ``EsnTorch`` library. It describes the implementation of a **Deep Echo State Network (Deep ESN)** for text classification on the **TREC-6** dataset (question classification).

The instantiation, training and evaluation of a Deep ESN for text classification is similar to that of a calssical ESN. It is achieved via the following steps:
1. Import libraries and modules
2. Load and prepare data
3. Instantiate the model:
    - specify parameters
    - specify learning algorithm
    - warm up
4. Train
5. Evaluate

## Librairies

In [1]:
# !pip install transformers==4.8.2
# !pip install datasets==1.7.0
# !pip install tqdm

In [2]:
# To enable progress bars in jupyter:

# pip install ipywidgets
# jupyter nbextension enable --py widgetsnbextension
# conda install -c conda-forge nodejs
# jupyter labextension install @jupyter-widgets/jupyterlab-manager

In [3]:
# Comment this if library is installed!
import os
import sys
sys.path.insert(0, os.path.abspath(".."))

In [4]:
from tqdm.notebook import tqdm_notebook
from sklearn.metrics import classification_report

import torch

from datasets import load_dataset, Dataset, concatenate_datasets

from transformers import AutoTokenizer
from transformers.data.data_collator import DataCollatorWithPadding

import esntorch.core.reservoir as res
import esntorch.core.learning_algo as la
import esntorch.core.merging_strategy as ms
import esntorch.core.esn as esn

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
# Set device (cpu or gpu if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Load and prepare data

### Load and tokenize data

In [7]:
# Custom functions for loading and preparing data
def tokenize(sample):
    """Tokenize sample"""
    
    sample = tokenizer(sample['text'], truncation=True, padding=False, return_length=True)
    
    return sample
    
def load_and_prepare_dataset(dataset_name, split, cache_dir):
    """
    Load dataset from the datasets library of HuggingFace.
    Tokenize and add length.
    """
    
    # Load dataset
    dataset = load_dataset(dataset_name, split=split, cache_dir=CACHE_DIR)
    
    # Rename label column for tokenization purposes (use 'label-fine' for fine-grained labels)
    dataset = dataset.rename_column('label-coarse', 'labels')
    
    # Tokenize data
    dataset = dataset.map(tokenize, batched=True)
    dataset = dataset.rename_column('length', 'lengths')
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels', 'lengths'])
    
    return dataset

In [8]:
# Load BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Load and prepare data
CACHE_DIR = 'cache_dir/' # put your path here

full_dataset = load_and_prepare_dataset('trec', split=None, cache_dir=CACHE_DIR)
train_dataset = full_dataset['train'].sort("lengths")
test_dataset = full_dataset['test'].sort("lengths")

# Create dict of all datasets
dataset_d = {
    'train': train_dataset,
    'test': test_dataset
    }

Using custom data configuration default
Reusing dataset trec (cache_dir/trec/default/1.1.0/751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9)
Parameter 'function'=<function tokenize at 0x7f2ef561ad30> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [9]:
dataset_d

{'train': Dataset({
     features: ['attention_mask', 'input_ids', 'label-fine', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 5452
 }),
 'test': Dataset({
     features: ['attention_mask', 'input_ids', 'label-fine', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 500
 })}

### Create dataloaders

In [10]:
# Create dict of dataloaders

dataloader_d = {}

for k, v in dataset_d.items():
    dataloader_d[k] = torch.utils.data.DataLoader(v, batch_size=256, 
                                                  collate_fn=DataCollatorWithPadding(tokenizer))

In [11]:
dataloader_d

{'train': <torch.utils.data.dataloader.DataLoader at 0x7f2ef564ce50>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7f2eecd32e20>}

## Instanciate the model

### Deep ESN

For Deep ESNs, set the parameter ``deep``to ``True``, then customize the number of layers (i.e., reservoirs) by specifying the parameter ``nb_layers``. Each of the other parameters (like ``dim``, ``distribution``, ``spectal radius``, etc.), can be specified in two ways:
- list of values: in this case, the successive layers are built according to the successive values of the parameter in the list;
- single value: in this case, the successive layers are all built according to the same value of the parameter.
Please refer to the documentation for further details.

In [12]:
# Deep ESN parameters
esn_params = {
            'embedding': 'bert-base-uncased',
#            'input_dim': 768,  # dim of BERT encoding!
            'dim': [500, 400, 300],      # *** list of dims for the successive layers ***
            'sparsity': 0.9,
            'spectral_radius': 0.9,
            'leaking_rate': 0.5,
            'activation_function': 'tanh',
            'bias_scaling': 0.1,
            'input_scaling': 0.1,
            'mean': 0.0,
            'std': 1.0,     
            'learning_algo': None,       # initialzed below
            'criterion': None,           # initialzed below (only for learning algos trained with SGD)
            'optimizer': None,           # initialzed below (only for learning algos trained with SGD)
            'merging_strategy': 'mean',
            'bidirectional': False,      # True, False
            'mode' : 'recurrent_layer',  # 'no_layer', 'linear_layer', 'recurrent_layer'
            'deep' : True,               # *** Deep ESN ***
            'nb_layers' : 3,             # *** 3 layers ***
            'device': device,
            'seed': 42
            }

# Instantiate the ESN
ESN = esn.EchoStateNetwork(**esn_params)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model downloaded: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model downloaded: bert-base-uncased


### Learning algorithm
Choose your learning algo by un-commenting its associated cell.

**Direct methods:**
To be used with the following learning algos:
- ``RidgeRegression``
- ``RidgeRegression_skl``
- ``LinearSVC``
- ``LogisticRegression_skl``

In [13]:
# ESN.learning_algo = la.RidgeRegression(alpha=10.0)

In [14]:
# ESN.learning_algo = la.RidgeRegression_skl(alpha=10.0)

In [15]:
# ESN.learning_algo = la.LinearSVC(C=1.0)

In [16]:
# ESN.learning_algo = la.LogisticRegression_skl()

**Gradient descent methods:**
To be used with the following learning algos:
- ``LogisticRegression`` (our implementation)
- ``DeepNN`` (our implementation)

In [17]:
input_dim = sum([layer.dim for layer in ESN.layer.layers])

if esn_params['bidirectional']:
    input_dim *= 2

In [18]:
ESN.learning_algo = la.LogisticRegression(input_dim=input_dim, output_dim=6)

In [19]:
# ESN.learning_algo = la.DeepNN([input_dim, 512, 256, 6])

In [20]:
# Needs criterion and otpimizer

ESN.criterion = torch.nn.CrossEntropyLoss()
ESN.optimizer = torch.optim.Adam(ESN.learning_algo.parameters(), lr=0.01)

### Warm up

In [21]:
# Put model on device
ESN = ESN.to(device)

In [22]:
if isinstance(ESN.layer, res.LayerRecurrent):
    ESN.warm_up(dataset_d['train'].select(range(10)))

## Training

For **direct methods**, the parameters ``epochs`` and ``iter_steps`` are ignored. 

In [27]:
ESN.fit(dataloader_d["train"], epochs=3, iter_steps=50)

Performing gradient descent...


Iteration: 50 Loss: 0.6930756568908691


Training complete.


[0.6930756568908691]

**Gradient descent methods:**
To be used with the following learning algos:
- ``LogisticRegression``
- ``DeepNN``

## Evaluation

In [24]:
# Train predictions and accuracy
train_pred, train_acc = ESN.predict(dataloader_d["train"], verbose=False)
train_acc

47.96404988994864

In [25]:
# Test predictions
test_pred, test_acc = ESN.predict(dataloader_d["test"], verbose=False)
test_acc

46.2

In [26]:
# Test classification report
print(classification_report(test_pred.tolist(), 
                            dataset_d['test']['labels'].tolist(), 
                            digits=4))

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000         0
           1     0.9787    0.2813    0.4371       327
           2     0.0000    0.0000    0.0000         0
           3     0.1077    1.0000    0.1944         7
           4     0.5752    0.8904    0.6989        73
           5     0.8272    0.7204    0.7701        93

    accuracy                         0.4620       500
   macro avg     0.4148    0.4820    0.3501       500
weighted avg     0.8794    0.4620    0.5338       500



/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
